<div align="center">
    <h1>Escuela Politécnica Nacional</h1>
    <h3>Recuperación de información</h3>
</div>


*Taller 06:* Base de datos Vectoriales


*Integrantes:*  
- Vickiann Jiménez
- Gabriela Salazar  
- Jostin Vega  



### Parte 1: Recuperación con TF-IDF

#### Cargar los datos en Python

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Cargar el dataset
df = pd.read_csv('../data/wiki_movie_plots_deduped.csv')
# Seleccionar las columnas necesarias
df = df[['Title', 'Plot']]
print(df.head())


                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                                Plot  
0  A bartender is working at a saloon, serving dr...  
1  The moon, painted with a smiling face hangs ov...  
2  The film, just over a minute long, is composed...  
3  Lasting just 61 seconds and consisting of two ...  
4  The earliest known adaptation of the classic f...  


#### Configurar TF-IDF:


In [ ]:
# Configurar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')

# Crear la matriz TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['Plot'])


#### Realizar consultas:



In [ ]:
# Función para realizar consultas con TF-IDF
def search_tfidf(query, top_k=5):
    """
    Realiza una búsqueda en los documentos utilizando la representación TF-IDF.

    Parámetros:
    query : str
        La consulta en formato de texto que el usuario desea buscar en los documentos.
    top_k : int, opcional
        El número de resultados más relevantes que se devolverán (por defecto es 5).

    Retorna:
    pd.DataFrame
        Un DataFrame con los títulos, tramas y puntajes de similitud de los documentos más relevantes encontrados.
        Las columnas del DataFrame incluyen:
        - 'Title': El título del documento.
        - 'Plot': La trama del documento.
        - 'Score': La similitud de coseno entre la consulta y el documento.
    """
    query_vec = vectorizer.transform([query])  # Vectorizar la consulta
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()  # Calcular similitudes
    top_indices = similarities.argsort()[-top_k:][::-1]  # Obtener los índices de los más similares
    results = df.iloc[top_indices][['Title', 'Plot']].copy()
    results['Score'] = similarities[top_indices]
    return results

In [ ]:
# Ejemplo de consulta
query = "dinosaurs"
results = search_tfidf(query)
print(results)


                                Title  \
12103  We're Back! A Dinosaur's Story   
12568                    Theodore Rex   
12989                      Future War   
21252            The Dinosaur Project   
31128                 Adhisaya Ulagam   

                                                    Plot     Score  
12103  In present-day New York City, an Eastern blueb...  0.467935  
12568  In an alternate futuristic society where human...  0.381369  
12989  Future War begins aboard a spaceship undergoin...  0.282999  
21252  A group of explorers from the British Cryptozo...  0.261480  
31128  Professor Neelakantan (J. Livingston) is an ma...  0.241712  


### Parte 2: Recuperación con BM25

#### Configurar Elasticsearch:

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pandas as pd

In [ ]:
# Conectar a Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [ ]:
# Crear índice
index_name = "movie_plots"

# Eliminar el índice si ya existe
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' eliminado.")

# Crear un nuevo índice
es.indices.create(index=index_name)
print(f"Índice '{index_name}' creado.")

Índice 'movie_plots' eliminado.
Índice 'movie_plots' creado.


In [ ]:
# Función para indexar datos con el índice original
def index_data():
    """
    Genera un conjunto de documentos formateados para indexarlos en Elasticsearch.

    Parámetros:
    - df: pd.DataFrame
        DataFrame que contiene los datos que se indexarán, con las columnas 'Title' (título) y 'Plot' (trama).
    - index_name: str
        Nombre del índice en Elasticsearch donde se guardarán los documentos.

    Retorna:
    generator
        Generador que produce un diccionario por cada fila del DataFrame, estructurado en el formato esperado por Elasticsearch.
    """

    for idx, row in df.iterrows():
        yield {
            "_index": index_name,
            "_source": {
                "Index": int(idx),  # Guardar el índice original como entero
                "Title": row['Title'],
                "Plot": row['Plot']
            }
        }

# Indexar los documentos
bulk(es, index_data())
print("Indexación completada.")

Indexación completada.


#### Realizar consultas:

In [ ]:
# Función para realizar consultas con BM25
def search_bm25(query, top_k=5):
    """
    Realiza una consulta en Elasticsearch utilizando el modelo BM25 para recuperar documentos relevantes.

    Parámetros:
    query : str
        La consulta de búsqueda. Es el texto que se usará para encontrar documentos relevantes en la colección.
    top_k : int, opcional
        Número de resultados más relevantes a recuperar (por defecto es 5).

    Retorna:
    pd.DataFrame
        Un DataFrame que contiene los documentos recuperados, con las siguientes columnas:
        - Index: Índice original del documento en el DataFrame `df`.
        - Title: Título del documento.
        - Plot: Trama del documento.
        - Score: Puntaje de relevancia calculado por BM25.
    """
    # Consulta Elasticsearch usando BM25
    query_body = {
        "query": {
            "match": {
                "Plot": query
            }
        },
        "size": top_k  # Recuperar los top_k resultados
    }

    response = es.search(index=index_name, body=query_body)

    # Procesar los resultados en un DataFrame
    results = []
    for hit in response['hits']['hits']:
        results.append({
            "Index": hit['_source']['Index'],  # Índice original
            "Title": hit['_source']['Title'],
            "Plot": hit['_source']['Plot'],
            "Score": hit['_score']
        })

    # Crear un DataFrame
    df_results = pd.DataFrame(results).set_index("Index")  # Establecer el índice original como índice del DataFrame
    return df_results

In [ ]:
# Ejemplo de consulta
query = "dinosaurs"
results = search_bm25(query)
print(results)

                                Title  \
Index                                   
12103  We're Back! A Dinosaur's Story   
12568                    Theodore Rex   
12989                      Future War   
4803                   Unknown Island   
31128                 Adhisaya Ulagam   

                                                    Plot      Score  
Index                                                                
12103  In present-day New York City, an Eastern blueb...  12.856184  
12568  In an alternate futuristic society where human...  11.746374  
12989  Future War begins aboard a spaceship undergoin...  11.591837  
4803   Adventure-seeker Ted Osborne (Phillip Reed) an...  11.046180  
31128  Professor Neelakantan (J. Livingston) is an ma...  10.903929  


### Parte 3: Recuperación con FAISS

#### Configurar FAISS:

In [ ]:
pip install sentence-transformers faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm  # Para mostrar barra de progreso
import os
import faiss

In [ ]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
# Cargar modelo preentrenado
model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('paraphrase-MiniLM-L3-v2')  # Modelo más rápido

# Dividir el trabajo en lotes
batch_size = 64
plots = df['Plot'].tolist()

# Inicializar lista para almacenar los embeddings
embeddings = []

print("Generating embeddings...")

# Procesar los datos en lotes y mostrar el progreso
for i in tqdm(range(0, len(plots), batch_size), desc="Processing batches"):
    batch = plots[i:i + batch_size]
    batch_embeddings = model.encode(batch)  # Generar embeddings para el lote
    embeddings.extend(batch_embeddings)  # Agregar al resultado final

# Convertir los embeddings a un formato NumPy
embeddings = np.array(embeddings)

print(f"Shape of embeddings: {embeddings.shape}")


Generating embeddings...


Processing batches: 100%|██████████| 546/546 [43:53<00:00,  4.82s/it]

Shape of embeddings: (34886, 384)


In [ ]:
# Crear el índice FAISS
dimension = embeddings.shape[1]  # Dimensión de los embeddings
index = faiss.IndexFlatL2(dimension)  # Usar métrica de distancia Euclidiana (L2)

# Agregar los embeddings al índice
index.add(embeddings)

print(f"Total vectors in the index: {index.ntotal}")


Total vectors in the index: 34886


#### Realizar consultas:

In [ ]:
# Función para realizar consultas con FAISS
def search_faiss(query, top_k=5):
    """
    Realiza una consulta en el índice FAISS para recuperar los documentos más relevantes basados en embeddings.

    Parámetros:
    query : str
        La consulta de búsqueda. Es el texto que se usará para generar un embedding y encontrar documentos relevantes.
    top_k : int, opcional
        Número de resultados más relevantes a recuperar (por defecto es 5).

    Retorna:
    pd.DataFrame
        Un DataFrame que contiene los documentos recuperados, con las siguientes columnas:
        - Title: Título del documento.
        - Plot: Trama del documento.
        - Distance: Distancia calculada en el espacio de embeddings (entre menor la distancia, mayor la relevancia).

    """
    # Generar embedding para la consulta
    query_vec = model.encode([query])

    # Buscar los vecinos más cercanos
    distances, indices = index.search(query_vec, top_k)

    # Obtener los resultados correspondientes
    results = df.iloc[indices.flatten()][['Title', 'Plot']].copy()
    results['Distance'] = distances.flatten()  # Agregar la distancia como métrica de relevancia
    return results

In [ ]:
# Ejemplo de consulta
query = "dinosaurs"
results = search_faiss(query)
print(results)

                                              Title  \
12016                                 Jurassic Park   
10697  Dinosaurs! – A Fun-Filled Trip Back in Time!   
7374                                 The Lost World   
12568                                  Theodore Rex   
10321               Baby: Secret of the Lost Legend   

                                                    Plot  Distance  
12016  Industrialist John Hammond and his bioengineer...  1.004112  
10697  The video—with beginning scenes filmed in 1987...  1.023169  
7374   Professor Challenger (Claude Rains), a famed b...  1.030121  
12568  In an alternate futuristic society where human...  1.031269  
10321  During an expedition into Central Africa, pale...  1.061895  


### Parte 4: Recuperación con ChromaDB

In [ ]:
pip install chromadb


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd

#### Configurar ChromaDB:

In [ ]:
chroma_client = chromadb.Client()

# Crear una colección para almacenar los embeddings
collection_name = "movie_plots_collection"
existing_collections = chroma_client.list_collections()

# Comprobar si la colección ya existe
if collection_name in existing_collections:
    chroma_client.delete_collection(name=collection_name)  # Si ya existe, eliminarla

collection = chroma_client.create_collection(name=collection_name)

#### Insertar documentos y embeddings:

In [ ]:
def insert_documents_to_chromadb(df, embeddings):
    """
    Inserta documentos, embeddings y metadatos en una colección de ChromaDB.

    Parámetros:
    df : pd.DataFrame
        DataFrame que contiene los datos de las películas, incluyendo las columnas:
        - 'Title': Título de la película.
        - 'Plot': Trama de la película.
    embeddings : np.ndarray
        Numpy array que contiene los embeddings generados para los documentos (plots).
        Cada fila corresponde al embedding de un documento en el DataFrame `df`.

    Retorno:
    None
        La función no retorna valores, pero inserta los datos en la colección especificada de ChromaDB.
    """
    ids = [str(idx) for idx in range(len(df))]  # IDs como strings
    documents = df['Plot'].tolist()  # Lista de textos (plots)
    metadatas = [{"Title": row['Title'], "Plot": row['Plot'], "Index": idx} for idx, row in df.iterrows()]  # Metadatos

    # Insertar en ChromaDB
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=embeddings.tolist(),  # Convertir a lista si es necesario
        metadatas=metadatas
    )
    print(f"Insertados {len(documents)} documentos en ChromaDB.")

In [ ]:
# Insertar los documentos con los embeddings
insert_documents_to_chromadb(df, embeddings)

Insertados 34886 documentos en ChromaDB.


#### Realizar consultas:

In [ ]:
# Función para realizar consultas con ChromaDB
def search_chromadb(query, top_k=5):
    """
    Realiza una consulta en ChromaDB para encontrar los documentos más relevantes
    basados en embeddings y devuelve los resultados más cercanos.

    Parámetros:
    query : str
        Texto de la consulta que el usuario desea buscar en la base de datos.
    top_k : int, opcional
        Número de resultados más cercanos que se deben recuperar (por defecto es 5).


    Retorno:
    pd.DataFrame
        DataFrame que contiene los resultados más relevantes, con las siguientes columnas:
        - "Index": Índice original del documento en el DataFrame de origen.
        - "Title": Título del documento.
        - "Plot": Trama del documento.
        - "Distance": Distancia del documento al embedding de la consulta (a menor distancia, mayor relevancia).

    """
    # Generar embedding para la consulta
    query_embedding = model.encode([query])[0]

    # Buscar los vectores más cercanos
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["metadatas", "distances"]
    )

    # Formatear los resultados en un DataFrame
    result_list = []
    for metadata, distance in zip(results["metadatas"][0], results["distances"][0]):
        result_list.append({
            "Index": metadata["Index"],
            "Title": metadata["Title"],
            "Plot": metadata["Plot"],
            "Distance": distance
        })

    df_results = pd.DataFrame(result_list).set_index("Index")
    return df_results

In [ ]:
# Ejemplo de consulta
query = "dinosaurs"
results = search_chromadb(query)
print(results)

                                              Title  \
Index                                                 
12016                                 Jurassic Park   
10697  Dinosaurs! – A Fun-Filled Trip Back in Time!   
7374                                 The Lost World   
12568                                  Theodore Rex   
10321               Baby: Secret of the Lost Legend   

                                                    Plot  Distance  
Index                                                               
12016  Industrialist John Hammond and his bioengineer...  1.004112  
10697  The video—with beginning scenes filmed in 1987...  1.023169  
7374   Professor Challenger (Claude Rains), a famed b...  1.030120  
12568  In an alternate futuristic society where human...  1.031269  
10321  During an expedition into Central Africa, pale...  1.061895  


### Parte 5: Comparación de Resultados

In [ ]:
from collections import Counter

In [ ]:
# Consulta y comparación
query = "dinosaurs"
top_k = 5

In [ ]:
# Realizar consultas y agregar columna "Method"
results_tfidf = search_tfidf(query, top_k).copy()
results_tfidf["Method"] = "TF-IDF"
results_tfidf["Original Index"] = results_tfidf.index

results_bm25 = search_bm25(query, top_k).copy()
results_bm25["Method"] = "BM25"
results_bm25["Original Index"] = results_bm25.index

results_faiss = search_faiss(query, top_k).copy()
results_faiss["Method"] = "FAISS"
results_faiss["Original Index"] = results_faiss.index

results_chromadb = search_chromadb(query, top_k).copy()
results_chromadb["Method"] = "ChromaDB"
results_chromadb["Original Index"] = results_chromadb.index

In [ ]:
# Concatenar los resultados en un único DataFrame
comparison_results = pd.concat([results_tfidf, results_bm25, results_faiss, results_chromadb], ignore_index=True)

In [ ]:
# Función para mostrar los resultados organizados por método
def display_results_by_method(df):
    methods = df["Method"].unique()
    for method in methods:
        print(f"\nResultados para el método: {method}\n")
        method_results = df[df["Method"] == method].copy()
        method_results.set_index("Original Index", inplace=True)
        # Ordenar por Score o Distance según corresponda
        if "Score" in method_results.columns:
            method_results = method_results.sort_values(by="Score", ascending=False)
        if "Distance" in method_results.columns:
            method_results = method_results.sort_values(by="Distance", ascending=True)
        print(method_results[["Title", "Score", "Distance"]])

In [ ]:
# Mostrar los resultados organizados por método
display_results_by_method(comparison_results)


Resultados para el método: TF-IDF

                                         Title     Score  Distance
Original Index                                                    
12103           We're Back! A Dinosaur's Story  0.467935       NaN
12568                             Theodore Rex  0.381369       NaN
12989                               Future War  0.282999       NaN
21252                     The Dinosaur Project  0.261480       NaN
31128                          Adhisaya Ulagam  0.241712       NaN

Resultados para el método: BM25

                                         Title      Score  Distance
Original Index                                                     
12103           We're Back! A Dinosaur's Story  12.856184       NaN
12568                             Theodore Rex  11.746374       NaN
12989                               Future War  11.591837       NaN
4803                            Unknown Island  11.046180       NaN
31128                          Adhisaya Ulagam  10.90

In [ ]:
def get_top_common_indices(query, top_k=25):
    """
    Identifica los índices más comunes entre los resultados devueltos por diferentes
    métodos de recuperación (TF-IDF, BM25, FAISS y ChromaDB) para una consulta específica.

    Parámetros:
    query : str
        El texto de la consulta que el usuario desea buscar en el corpus.
    top_k : int, opcional
        El número de resultados más relevantes que se deben recuperar de cada método (por defecto es 25).

    Retorno:
    list
        Lista de índices relevantes que aparecen en los resultados de al menos dos métodos diferentes.

    """
    # Obtener resultados de los métodos
    indices_tfidf = search_tfidf(query, top_k=top_k).index.tolist()
    indices_bm25 = search_bm25(query, top_k=top_k).index.tolist()
    indices_faiss = search_faiss(query, top_k=top_k).index.tolist()
    indices_chromadb = search_chromadb(query, top_k=top_k).index.tolist()

    # Combinar resultados
    combined_indices = indices_tfidf + indices_bm25 + indices_faiss + indices_chromadb

    # Contar frecuencia de aparición de los índices
    counter = Counter(combined_indices)

    # Seleccionar los más comunes
    relevant_indices = [idx for idx, count in counter.items() if count > 1]  # Índices que aparecen al menos en 2 métodos
    return relevant_indices

In [ ]:
# Crear automáticamente los documentos relevantes
queries = ["dinosaurs", "cyborg", "story", "war", "space"]
relevant_documents = {query: get_top_common_indices(query) for query in queries}

print(relevant_documents)

{'dinosaurs': [12103, 12568, 12989, 21252, 31128, 10321, 4803, 7342, 12390, 12016, 13054, 12644, 12484, 5425, 5536, 33672, 33377, 33398, 10697, 7374, 13637, 17914, 10994, 16998, 15699, 33090, 33180, 15014, 33050, 7175, 19427, 12160, 14126], 'cyborg': [12989, 33064, 33072, 12838, 23638, 34434, 33521, 23660, 33017, 33018, 12407, 17621, 12170, 16582, 11849, 33175, 33102, 8074, 11240, 14466, 33408, 33435, 17071, 32930, 32944, 33088, 16748, 21326, 11623, 33716, 33278, 10298, 13509, 11269, 33200, 21735, 33489, 33228, 33863, 33397, 33148], 'story': [24974, 25, 29178, 23427, 22842, 26969, 22113, 28479, 29688, 23887, 28963, 28999, 28952, 34851, 30668, 29207, 31594, 29235, 17238, 22964, 30868, 20159, 34796], 'war': [18511, 18893, 18810, 6560, 5630, 33671, 24929, 5569, 10421, 5623, 18188, 20384, 3814, 31611, 3642, 3402, 18090, 18881, 30179, 18125, 16451, 10667, 18430, 21719, 7595, 22102, 7578, 34323, 34244, 11794, 9320, 17378, 14114, 3473, 33099], 'space': [33756, 21592, 7121, 6174, 12181, 13133,

In [ ]:
def calculate_relevance_metrics(query, results_df, relevant_indices):
    """
    Calcula las métricas de relevancia (Precisión y Recall) para una consulta específica
    basándose en los resultados recuperados y los índices de documentos relevantes.

    Parámetros:
    query : str
        El texto de la consulta que se está evaluando.
    results_df : pandas.DataFrame
        DataFrame que contiene los resultados recuperados por un método de búsqueda.
        Debe tener los índices de los documentos como `Index`.
    relevant_indices : list
        Lista de índices que representan los documentos relevantes para la consulta.

    Retorno:
    tuple
        Una tupla que contiene:
        - `precision` (float): Proporción de documentos relevantes sobre los recuperados.
        - `recall` (float): Proporción de documentos relevantes recuperados sobre el total de documentos relevantes.

    """
    retrieved_indices = results_df.index.tolist()  # Índices de los documentos recuperados
    relevant_retrieved = set(retrieved_indices) & set(relevant_indices)  # Intersección: relevantes recuperados

    precision = len(relevant_retrieved) / len(retrieved_indices) if retrieved_indices else 0  # Qué tan precisos son
    recall = len(relevant_retrieved) / len(relevant_indices) if relevant_indices else 0  # Qué tan completos son

    return precision, recall


In [ ]:
def compare_methods(queries, relevant_documents):
    """
    Compara las métricas de precisión y recall de diferentes métodos de recuperación (TF-IDF, BM25, FAISS, ChromaDB)
    para un conjunto de consultas y documentos relevantes.

    Parámetros:
    queries : list
        Lista de consultas (strings) a evaluar.
    relevant_documents : dict
        Diccionario donde las claves son las consultas (strings) y los valores son listas de índices relevantes para cada consulta.

    Retorno:
    pandas.DataFrame
        DataFrame con las métricas de precisión y recall para cada consulta y método.
        Contiene las columnas:
        - `Query`: La consulta evaluada.
        - `Method`: El método de recuperación evaluado (e.g., TF-IDF, BM25, FAISS, ChromaDB).
        - `Precision`: La precisión obtenida para ese método y consulta.
        - `Recall`: El recall obtenido para ese método y consulta.
    """
    comparison_metrics = []

    for query in queries:
        print(f"\nConsulta: {query}")
        relevant_indices = relevant_documents[query]

        # Ejecutar consultas en cada método
        tfidf_results = search_tfidf(query, top_k=10)
        bm25_results = search_bm25(query, top_k=10)
        faiss_results = search_faiss(query, top_k=10)
        chromadb_results = search_chromadb(query, top_k=10)

        # Calcular métricas de relevancia para cada método
        tfidf_precision, tfidf_recall = calculate_relevance_metrics(query, tfidf_results, relevant_indices)
        bm25_precision, bm25_recall = calculate_relevance_metrics(query, bm25_results, relevant_indices)
        faiss_precision, faiss_recall = calculate_relevance_metrics(query, faiss_results, relevant_indices)
        chromadb_precision, chromadb_recall = calculate_relevance_metrics(query, chromadb_results, relevant_indices)

        # Guardar los resultados
        comparison_metrics.append({
            "Query": query,
            "Method": "TF-IDF",
            "Precision": tfidf_precision,
            "Recall": tfidf_recall
        })
        comparison_metrics.append({
            "Query": query,
            "Method": "BM25",
            "Precision": bm25_precision,
            "Recall": bm25_recall
        })
        comparison_metrics.append({
            "Query": query,
            "Method": "FAISS",
            "Precision": faiss_precision,
            "Recall": faiss_recall
        })
        comparison_metrics.append({
            "Query": query,
            "Method": "ChromaDB",
            "Precision": chromadb_precision,
            "Recall": chromadb_recall
        })

    return pd.DataFrame(comparison_metrics)

In [ ]:
# Ejecutar la comparación
metrics_df = compare_methods(queries, relevant_documents)
print(metrics_df)



Consulta: dinosaurs

Consulta: cyborg

Consulta: story

Consulta: war

Consulta: space
        Query    Method  Precision    Recall
0   dinosaurs    TF-IDF        1.0  0.303030
1   dinosaurs      BM25        1.0  0.303030
2   dinosaurs     FAISS        1.0  0.303030
3   dinosaurs  ChromaDB        1.0  0.303030
4      cyborg    TF-IDF        1.0  0.243902
5      cyborg      BM25        1.0  0.243902
6      cyborg     FAISS        1.0  0.243902
7      cyborg  ChromaDB        1.0  0.243902
8       story    TF-IDF        0.3  0.130435
9       story      BM25        0.2  0.086957
10      story     FAISS        0.9  0.391304
11      story  ChromaDB        1.0  0.434783
12        war    TF-IDF        0.7  0.200000
13        war      BM25        0.9  0.257143
14        war     FAISS        0.9  0.257143
15        war  ChromaDB        1.0  0.285714
16      space    TF-IDF        0.9  0.243243
17      space      BM25        1.0  0.270270
18      space     FAISS        1.0  0.270270
19      spac